# Índice

- [Spark SQL](#sql)
    - [Dataframes](#df)
        - [Creación de Dataframes](#newdf)
        - [Operaciones con Dataframes](#operations)
            - [Transformaciones](#transforms)
            - [Acciones](#actions)

    - [SQL Queries](#queries)
    - [HIVE Queries](#hive)

<div id="sql" />

<div id="sql" />

# Spark SQL

[Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html) es uno de los módulos que forma parte del stack de Spark y que está destinado al procesado de datos estructurados. Las interfaces relacionadas con Spark SQL proporciona más información sobre los datos y la forma de computarlos que los RDD, dicha información se utiliza para realizar optimización que mejora el rendimiento.

Hay diversas formas de interactuar con Spark SQL, el API de los **Dataframes** y el de los **Datasets**. Todas ellas comparten la misma máquina de ejecución y es muy sencillo combinar distintos medios de interacción con Spark SQL de tal manera que los desarrolladores elijan en cada momento el que sea más cómodo.

Spark SQL proporciona objetos para el acceso y tratamiento de datos denominados **Dataframes** y un motor de consultas distribuido. Es capaz de localizar tablas y metadatos ya existentes en HIVE sin necesidad de realizar ningún trabajo extra (auqnue requiere cierto trabajo de configuración). También se puede interactuar con Spark SQL a través de línea de comandos mediante la herramienta *spak-sql* o utilizando JDBC/ODBC.

<div id="df" />

## Dataframes de Spark

Los dataframes de Spark equivalen a una tabla en una base de datos relacional o a un dataframe de python o R pero que almacena sus datos de manera distribuida y que cuenta con optimizaciones avanzadas orientadas a facilitar el análisis de datos a gran escala.

El dataframe de Spark SQL es equivalente al RDD en Spark Core pero con la diferencia fundamental de que los datos son estructurados. Para crear un dataframe necesitamos una clase *SparkContext*.



In [1]:
# Uno u otro

from pyspark.sql import SparkSession, SQLContext, HiveContext
from pyspark import SparkContext
#from pyspark import SparkConf

#SPARK_CONF = SparkConf().setMaster("yarn-clietn").setAppName("app_name")
#spark = SparkSession.builder.config(conf=SPARK_CONF).enableHiveSupport().getOrCreate()

sc = SparkContext()

spark = SparkSession(sc)
sqlc = SQLContext(sc)
hivec = HiveContext(sc)

Los Spark dataframes tienen una serie de características:

- Son inmutables: no se pueden cambiar una vez construidos
- Mantienen información de los planes de ejecución que permite recomputar lps datos perdidos de manera eficiente en caso de fallo
- Permiten realizar operaciones en colecciones de elementos en paralelo.

El ciclo de programación con los dataframes es muy similar al de los RDD:

1) Crear Dtaframe: desde dtos externos, colecciones o RDDs existentes.

2) Transformar los dataframes en nuevos dataframes.

3) *Cachear* algún dataframe para reusarlos posteriormente

4) Ejecutar acciones que corran en paralelo y produzcan resultados.

Spark SQL soporta los siguientes tipos de datos:


![alt text](../images/data_type.png "Tipos de datos")


Para importar todos los tipos de datos y poderlos usar en nuestros programas:

In [2]:
from pyspark.sql.types import *

<div id="newdf" />

### Creación de dataframes

- **A partir de una lista python.**

In [3]:
iris_data = [(4.3, 3.0, 1.1, 0.1, "Iris-setosa"), (7.0, 3.2, 4.7, 1.4, "Iris-versicolor"), (6.9, 3.2, 5.7, 2.3, "Iris-virginica")]
print("Iris Data", iris_data)
print(" ")
df = spark.createDataFrame(iris_data, ["sepal_length", "sepal_width", "petal_length", "petal_width", "specie"])
df.show()
df.collect()

Iris Data [(4.3, 3.0, 1.1, 0.1, 'Iris-setosa'), (7.0, 3.2, 4.7, 1.4, 'Iris-versicolor'), (6.9, 3.2, 5.7, 2.3, 'Iris-virginica')]
 
+------------+-----------+------------+-----------+---------------+
|sepal_length|sepal_width|petal_length|petal_width|         specie|
+------------+-----------+------------+-----------+---------------+
|         4.3|        3.0|         1.1|        0.1|    Iris-setosa|
|         7.0|        3.2|         4.7|        1.4|Iris-versicolor|
|         6.9|        3.2|         5.7|        2.3| Iris-virginica|
+------------+-----------+------------+-----------+---------------+



[Row(sepal_length=4.3, sepal_width=3.0, petal_length=1.1, petal_width=0.1, specie='Iris-setosa'),
 Row(sepal_length=7.0, sepal_width=3.2, petal_length=4.7, petal_width=1.4, specie='Iris-versicolor'),
 Row(sepal_length=6.9, sepal_width=3.2, petal_length=5.7, petal_width=2.3, specie='Iris-virginica')]

- **A partir de un dataframe de pandas.**

In [4]:
import pandas as pd

pandas_df = pd.DataFrame(iris_data)
spark_df = spark.createDataFrame(pandas_df)
spark_df.show()

+---+---+---+---+---------------+
|  0|  1|  2|  3|              4|
+---+---+---+---+---------------+
|4.3|3.0|1.1|0.1|    Iris-setosa|
|7.0|3.2|4.7|1.4|Iris-versicolor|
|6.9|3.2|5.7|2.3| Iris-virginica|
+---+---+---+---+---------------+



- **A partir de un RDD.** Spark SQL soporta dos métodos diferentes para convertir RDD ya existentes en dataframes. El primer método usa **reflection** para inferir el esquema de un RDD que contiene tipos específicos de objetos. Esta aproximación utiliza un código muy conciso y funciona bien cuando el esquema ya es conocido al programar la aplicación Spark.

In [5]:
from pyspark.sql import Row

iris = sc.textFile("../data/iris.csv")

# Eliminamos cabeceras
headers = iris.first()
rdd = iris.filter(lambda line: line != headers)

rdd = rdd.map(lambda x: x.split(","))

# Metemos cada uno de los campos separados por coma en un objeto tipo fila (Row) y le damos un nombre
rdd = rdd.map(lambda x: Row(ID=int(x[0]), sepal_length=float(x[1]), sepal_width=float(x[2]), petal_length=float(x[3]), petal_width=float(x[4]), specie=x[5]))
#print(rdd.take(5))

irisDF = spark.createDataFrame(rdd)
irisDF.show(5)

+---+------------+-----------+------------+-----------+-----------+
| ID|petal_length|petal_width|sepal_length|sepal_width|     specie|
+---+------------+-----------+------------+-----------+-----------+
|  1|         1.4|        0.2|         5.1|        3.5|Iris-setosa|
|  2|         1.4|        0.2|         4.9|        3.0|Iris-setosa|
|  3|         1.3|        0.2|         4.7|        3.2|Iris-setosa|
|  4|         1.5|        0.2|         4.6|        3.1|Iris-setosa|
|  5|         1.4|        0.2|         5.0|        3.6|Iris-setosa|
+---+------------+-----------+------------+-----------+-----------+
only showing top 5 rows



La segunda forma de crear un dataframe a partir de un RDD se conoce como **programática** y es especialmente útil cuando el diccionario de campos no se conoce antes de la creación del propio dataframe: por ejemplo cuando la estructura de campos está codificada en una cadena de caracteres y hay que leerlos desde ahí.

Los pasos a seguir en este tipo de creaciones es el siguiente:

1) Crear un RDD de tuplas o listas desde el RDD original (habitualmente usando la función "map"). 

2) Crear un esquema representado por un tipo StructType que coincida con la estructura de las tuplas o listas del RDD del punto anterior.

3) Aplicar el esquema RDD vía método createDataframe

In [6]:
#from pyspark.sql.types import *

rdd = iris.filter(lambda line: line != headers)

rdd = rdd.map(lambda x: x.split(","))
rdd = rdd.map(lambda x: (int(x[0]), float(x[1]), float(x[2]), float(x[3]), float(x[4]), x[5]))

# Definimos el nombre y tipo de campos para nuestro dataframe
fields = [StructField("ID", IntegerType(), True), StructField("sepal_length", FloatType(), True), StructField("sepal_width", FloatType(), True), StructField("petal_length", FloatType(), True), StructField("petal_width", FloatType(), True), StructField("specie", StringType(), True)]

# Definimos el esquema con los tipos de campos creados
schema = StructType(fields)

irisDF2 = spark.createDataFrame(rdd, schema)
irisDF2.show(5)

+---+------------+-----------+------------+-----------+-----------+
| ID|sepal_length|sepal_width|petal_length|petal_width|     specie|
+---+------------+-----------+------------+-----------+-----------+
|  1|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|  2|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|  3|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|  4|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|  5|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
+---+------------+-----------+------------+-----------+-----------+
only showing top 5 rows



- **Es posible crear un RDD a partir de un dataframe.**

In [7]:
irisRDD = irisDF2.select("sepal_length", "sepal_width", "specie").rdd
irisRDD.take(5)

[Row(sepal_length=5.099999904632568, sepal_width=3.5, specie='Iris-setosa'),
 Row(sepal_length=4.900000095367432, sepal_width=3.0, specie='Iris-setosa'),
 Row(sepal_length=4.699999809265137, sepal_width=3.200000047683716, specie='Iris-setosa'),
 Row(sepal_length=4.599999904632568, sepal_width=3.0999999046325684, specie='Iris-setosa'),
 Row(sepal_length=5.0, sepal_width=3.5999999046325684, specie='Iris-setosa')]

- **A partir de un fichero JSON.** El fichero tiene que cumplir ciertos requisitos: cada línea debe contener un objeto separado, válido y autocontenido, es decir, un fichero JSON convencional del tipo multilínea fallará casi con el 100% de seguridad.

In [8]:
#from pyspark.sql.types import *
iris_jsonDF = spark.read.format("json").load("../data/iris.json")
iris_jsonDF.show(5)

+---------------+-----------+----------+-----------+----------+-------+
|_corrupt_record|petalLength|petalWidth|sepalLength|sepalWidth|species|
+---------------+-----------+----------+-----------+----------+-------+
|              [|       null|      null|       null|      null|   null|
|           null|        1.4|       0.2|        5.1|       3.5| setosa|
|           null|        1.4|       0.2|        4.9|       3.0| setosa|
|           null|        1.3|       0.2|        4.7|       3.2| setosa|
|           null|        1.5|       0.2|        4.6|       3.1| setosa|
+---------------+-----------+----------+-----------+----------+-------+
only showing top 5 rows



- **A partir de un fichero csv.**

In [9]:
#from pyspark.sql.types import *

#iris_csvDF = spark.read.format("csv").options(header=True).load("../data/iris.csv")
#iris_csvDF = spark.read.format("csv").options(header=True, inferSchema=True).load("../data/iris.csv")
iris_csvDF = spark.read.format("csv").options(header=True).load("../data/iris.csv", schema=schema)
iris_csvDF.show(3)

+---+------------+-----------+------------+-----------+-----------+
| ID|sepal_length|sepal_width|petal_length|petal_width|     specie|
+---+------------+-----------+------------+-----------+-----------+
|  1|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|  2|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|  3|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
+---+------------+-----------+------------+-----------+-----------+
only showing top 3 rows



- **A partir de una tabla HIVE.** Siempre que tengamos activado el soporte HIVE activado (*enableHiveSupport()*).

In [10]:
# Debe existir en Hive previamente la tabla "iris-tabla-hive" en la base de datos "test"

#irisHIVE = spark.table("test.iris-tabla-hive")

<div id="operations" />

### Operaciones con dataframes

Al igual que con los RDD, con los dataframes podemos realizar dos tipos de operaciones: **transformaciones** y **acciones**. También al igual que con los RDD, las transformaciones son *lazy*, es decir, que no se computan inmediatamente sino que son modifiados de forma efectiva cuando se aplica una **acción** sobre ellos.

<div id="transforms" />

#### Transformaciones

El resultado de una **transformación** es otro dataframe.

Algunos ejemplos de transformaciones sobre un dataframe son las siguientes:

In [11]:
irisSpecie = irisDF.select("specie")
irisSpecie.show(2)

+-----------+
|     specie|
+-----------+
|Iris-setosa|
|Iris-setosa|
+-----------+
only showing top 2 rows



In [12]:
irisSpecie2 = irisDF.select(irisDF.petal_length, irisDF.specie)
irisSpecie2.show(2)

+------------+-----------+
|petal_length|     specie|
+------------+-----------+
|         1.4|Iris-setosa|
|         1.4|Iris-setosa|
+------------+-----------+
only showing top 2 rows



In [13]:
irisSpecie3 = irisDF.select((irisDF.petal_length + 1).alias("petal_length + 1"), irisDF.specie)
irisSpecie3.show(2)

+----------------+-----------+
|petal_length + 1|     specie|
+----------------+-----------+
|             2.4|Iris-setosa|
|             2.4|Iris-setosa|
+----------------+-----------+
only showing top 2 rows



In [14]:
irisSinSpecie = irisDF.drop(irisDF.specie)
irisSinSpecie.show(2)

+---+------------+-----------+------------+-----------+
| ID|petal_length|petal_width|sepal_length|sepal_width|
+---+------------+-----------+------------+-----------+
|  1|         1.4|        0.2|         5.1|        3.5|
|  2|         1.4|        0.2|         4.9|        3.0|
+---+------------+-----------+------------+-----------+
only showing top 2 rows



***filter(func)*** devuelve un dataframe resultado de la selección de las filas del dataframe original en las que la aplicación de la función retorna "True" (se cumple la condición)

In [15]:
print("Número de filas antes del filtrado: ", irisDF.count())

irisFiltrado = irisDF.filter(irisDF.sepal_length > 5.0)

print("Número de filas después del filtrado: ", irisFiltrado.count())

Número de filas antes del filtrado:  150
Número de filas después del filtrado:  118


***distinct()*** devuelve un dataframe que contiene solamente las filas distintas del dataframe original.

In [16]:
print("Número de filas totales: ", irisDF.count())
print("Número de filas NO repetidas: ", irisDF.distinct().count())

Número de filas totales:  150
Número de filas NO repetidas:  150


***orderby(cols, kw)*** crea un nuevo dataframe ordenado por las columnas especificadas (cols) y en el sentido indicado (kw).

***sort(cols, kw)*** tiene el mismo comportamiento que orderby.

In [17]:
irisDF.sort("specie", ascending = False).show(2)

+---+------------+-----------+------------+-----------+--------------+
| ID|petal_length|petal_width|sepal_length|sepal_width|        specie|
+---+------------+-----------+------------+-----------+--------------+
|103|         5.9|        2.1|         7.1|        3.0|Iris-virginica|
|101|         6.0|        2.5|         6.3|        3.3|Iris-virginica|
+---+------------+-----------+------------+-----------+--------------+
only showing top 2 rows



***explode(col)*** retorna un nuevo dataframe en el que se crea una nueva fila por cada uno de los elementos de una columna que contenga un dato tipo array o map.

In [18]:
#from pyspark.sql import Row
from pyspark.sql.functions import explode

data = [Row(specie="Virginica", sepal_length_list=[0.1, 2.3, 3.1, 4.0])]

irisE = spark.createDataFrame(data)
irisE.show()

irisExplode = irisE.select(irisE.specie, explode(irisE.sepal_length_list).alias("sepal_length"))
irisExplode.show()

+--------------------+---------+
|   sepal_length_list|   specie|
+--------------------+---------+
|[0.1, 2.3, 3.1, 4.0]|Virginica|
+--------------------+---------+

+---------+------------+
|   specie|sepal_length|
+---------+------------+
|Virginica|         0.1|
|Virginica|         2.3|
|Virginica|         3.1|
|Virginica|         4.0|
+---------+------------+



***udf()*** user-defined-functions. Además de las funciones que podemos crear nosotros mismos, Spark cuenta con una lista muy extensa de funciones predefinidas.

In [19]:
#from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

masUno = udf(lambda x: x + 1, FloatType())

irisDFMasUno = irisDF.select(irisDF.specie, masUno(irisDF.sepal_length).alias("sepal_length + 1"))
irisDFMasUno.show(3)

+-----------+----------------+
|     specie|sepal_length + 1|
+-----------+----------------+
|Iris-setosa|             6.1|
|Iris-setosa|             5.9|
|Iris-setosa|             5.7|
+-----------+----------------+
only showing top 3 rows



También podemos hacer transformaciones de agrupación sobre dataframes:

***groupby()*** agrupa el dataframe utilizando las columnas especificadas, pudiendo después aplicar operaciones de agregación sobre ellos.

***agg()*** calcula agregados (avg, max, min, sum y count) y retorna los resultados como un dataframe.

***count()*** cuenta el número de filas.

***avg() mean()*** calcula los valores medios de las columnas por cada grupo.

***max() min() sum()***

***pivot(col, values)*** pivota una columna del dataframe y calcula el agregado correspondiente.

In [20]:
from pyspark.sql.functions import avg

irisG = irisDF.groupby(irisDF.specie)

irisGCount = irisG.agg({"*": "count"})
irisGCount.show()

irisGAVG = irisG.agg(avg("sepal_length"), avg("sepal_width"))
irisGAVG.show()

+---------------+--------+
|         specie|count(1)|
+---------------+--------+
| Iris-virginica|      50|
|    Iris-setosa|      50|
|Iris-versicolor|      50|
+---------------+--------+

+---------------+-----------------+------------------+
|         specie|avg(sepal_length)|  avg(sepal_width)|
+---------------+-----------------+------------------+
| Iris-virginica|6.587999999999998|2.9739999999999998|
|    Iris-setosa|5.005999999999999|3.4180000000000006|
|Iris-versicolor|            5.936|              2.77|
+---------------+-----------------+------------------+



In [21]:
# Otra manera directamente sobre el dataframe

irisG.count().show()
irisG.avg().show()
irisG.avg("sepal_length").show()

+---------------+-----+
|         specie|count|
+---------------+-----+
| Iris-virginica|   50|
|    Iris-setosa|   50|
|Iris-versicolor|   50|
+---------------+-----+

+---------------+-------+-----------------+------------------+-----------------+------------------+
|         specie|avg(ID)|avg(petal_length)|  avg(petal_width)|avg(sepal_length)|  avg(sepal_width)|
+---------------+-------+-----------------+------------------+-----------------+------------------+
| Iris-virginica|  125.5|            5.552|             2.026|6.587999999999998|2.9739999999999998|
|    Iris-setosa|   25.5|            1.464|0.2439999999999999|5.005999999999999|3.4180000000000006|
|Iris-versicolor|   75.5|             4.26|1.3260000000000003|            5.936|              2.77|
+---------------+-------+-----------------+------------------+-----------------+------------------+

+---------------+-----------------+
|         specie|avg(sepal_length)|
+---------------+-----------------+
| Iris-virginica|6.587

Tipos de join: ***inner, outer, left_outer, right_outer, leftsemi***

In [22]:
#iris_data = [(4.3, 3.0, 1.1, 0.1, "Iris-setosa"), (7.0, 3.2, 4.7, 1.4, "Iris-versicolor"), (6.9, 3.2, 5.7, 2.3, "Iris-virginica")]
iris_col = [("Iris-setosa", "Naranja"), ("Iris-virginica", "Violeta")]

#df = spark.createDataFrame(iris_data, ["sepal_length", "sepal_width", "petal_length", "petal_width", "specie"])
iris_df = df
iris_col_df = spark.createDataFrame(iris_col, ["especie", "color"])

# Inner join
iris_df.join(iris_col_df, iris_df.specie == iris_col_df.especie, "inner").select("sepal_length", "sepal_width", "petal_length", "petal_width", "specie", "color").show()

# Left outer join
iris_df.join(iris_col_df, iris_df.specie == iris_col_df.especie, "left_outer").select("sepal_length", "sepal_width", "petal_length", "petal_width", "specie", "color").show()

+------------+-----------+------------+-----------+--------------+-------+
|sepal_length|sepal_width|petal_length|petal_width|        specie|  color|
+------------+-----------+------------+-----------+--------------+-------+
|         6.9|        3.2|         5.7|        2.3|Iris-virginica|Violeta|
|         4.3|        3.0|         1.1|        0.1|   Iris-setosa|Naranja|
+------------+-----------+------------+-----------+--------------+-------+

+------------+-----------+------------+-----------+---------------+-------+
|sepal_length|sepal_width|petal_length|petal_width|         specie|  color|
+------------+-----------+------------+-----------+---------------+-------+
|         6.9|        3.2|         5.7|        2.3| Iris-virginica|Violeta|
|         4.3|        3.0|         1.1|        0.1|    Iris-setosa|Naranja|
|         7.0|        3.2|         4.7|        1.4|Iris-versicolor|   null|
+------------+-----------+------------+-----------+---------------+-------+



Spark cuenta también con transformaciones de tipo estadístico y tratamiento de valores nulos o desconocidos:

***corr()*** obtiene el índice de correlación entre dos columnas del datraframe de tipo numérico. Aplicada sobre un dataframe devuelve un tipo numérico mientras que aplicada sobre las columnas con select() o add() devuelve otra columna.

***cov()*** similar a corr() pero calcula la covarianza.

***crosstab()*** calcula el número de pares de los elementos de dos columnas.

***freqItems()*** busca elementos frecuentes en dos columnas a partir de un número mínimo de apariciones.

***sampleBy()*** realiza una selección aleatoria de observaciones en base a una proporción prefijada de valores de una variable del dataframe.

In [23]:
from pyspark.sql.functions import count, corr, sum

print("Correlación sepal_petal_length: {0}".format(iris_csvDF.corr("sepal_length", "petal_length")))

iris_csvDF.select(corr("sepal_length", "petal_length").alias("corr_sepal_petal_length"), corr("sepal_width", "petal_width").alias("corr_sepal_petal_width")).show()

Correlación sepal_petal_length: 0.8717541656669878
+-----------------------+----------------------+
|corr_sepal_petal_length|corr_sepal_petal_width|
+-----------------------+----------------------+
|     0.8717541656669878|  -0.35654410088264576|
+-----------------------+----------------------+



***replace()*** actúa sobre observaciones con valores desconocidos.

***fill()** reemplaza valores nulos por otro valor.

***drop()*** borra las filas del dataframe que tienen valor null en las columnas que se decida.

In [24]:
iris_data = [(4.3, 3.0, 0.0, 0.1, "Iris-setosa"), (7.0, 3.2, 4.7, 1.4, "Iris-versicolor"), (6.9, 3.2, 5.7, 0.0, "Iris-virginica")]
iris_zero = spark.createDataFrame(iris_data, ["sepal_length", "sepal_width", "petal_length", "petal_width", "specie"])
iris_zero.show()
iris_zero.na.replace(0.0, 1.0, ["petal_length"]).show()

+------------+-----------+------------+-----------+---------------+
|sepal_length|sepal_width|petal_length|petal_width|         specie|
+------------+-----------+------------+-----------+---------------+
|         4.3|        3.0|         0.0|        0.1|    Iris-setosa|
|         7.0|        3.2|         4.7|        1.4|Iris-versicolor|
|         6.9|        3.2|         5.7|        0.0| Iris-virginica|
+------------+-----------+------------+-----------+---------------+

+------------+-----------+------------+-----------+---------------+
|sepal_length|sepal_width|petal_length|petal_width|         specie|
+------------+-----------+------------+-----------+---------------+
|         4.3|        3.0|         1.0|        0.1|    Iris-setosa|
|         7.0|        3.2|         4.7|        1.4|Iris-versicolor|
|         6.9|        3.2|         5.7|        0.0| Iris-virginica|
+------------+-----------+------------+-----------+---------------+



<div id="actions" />

#### Acciones

Hacen que Spark ejecute las transformaciones que ya se hayan programado anteriormente sobre los dataframes. Dicho de otro modo, son mecanismos para obtener resultados y mostrarlos o sacarlos fuera de spark.

***show() take() collect() printSchema() count() describe()***

In [25]:
df.describe().show()
df.printSchema()

+-------+-----------------+-------------------+------------------+------------------+--------------+
|summary|     sepal_length|        sepal_width|      petal_length|       petal_width|        specie|
+-------+-----------------+-------------------+------------------+------------------+--------------+
|  count|                3|                  3|                 3|                 3|             3|
|   mean|6.066666666666667| 3.1333333333333333|3.8333333333333335|1.2666666666666666|          null|
| stddev|1.530795000427338|0.11547005383792526| 2.419366308216541|1.1060440015358037|          null|
|    min|              4.3|                3.0|               1.1|               0.1|   Iris-setosa|
|    max|              7.0|                3.2|               5.7|               2.3|Iris-virginica|
+-------+-----------------+-------------------+------------------+------------------+--------------+

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true

<div id="queries" />

## Spark SQL Queries

El SQL que soportan los Spark dataframes permite realizar operaciones sobre tablas **temporales** creadas sobre los propios dataframes. Podemos realizar queries, añadir, modificar o borrar registros y crear, modificar o eliminar relaciones.

Para utilizar Spark SQL es necesario asociar una tabla temporal al dataframe mediante la aplicación de la función *registerTempTable()*. Es importante saber que los resultados de los *select()* son nuevos dataframes. Por último, el borrado de una tabla temporal no conlleva el borrado del dataframe a la que fue asignada.

In [26]:
irisDF.registerTempTable("iris_table")
spark.sql("show tables").show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|        |iris_table|       true|
+--------+----------+-----------+



In [27]:
spark.sql("select * from iris_table where sepal_length > 5 limit 3").show()

+---+------------+-----------+------------+-----------+-----------+
| ID|petal_length|petal_width|sepal_length|sepal_width|     specie|
+---+------------+-----------+------------+-----------+-----------+
|  1|         1.4|        0.2|         5.1|        3.5|Iris-setosa|
|  6|         1.7|        0.4|         5.4|        3.9|Iris-setosa|
| 11|         1.5|        0.2|         5.4|        3.7|Iris-setosa|
+---+------------+-----------+------------+-----------+-----------+



In [28]:
spark.sql("select specie, count(*) as num_specie from iris_table group by specie order by specie asc").show()

+---------------+----------+
|         specie|num_specie|
+---------------+----------+
|    Iris-setosa|        50|
|Iris-versicolor|        50|
| Iris-virginica|        50|
+---------------+----------+



In [29]:
spark.sql("drop table iris_table")
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



<div id="hive" />

## Queries sobre HIVE

Spark SQL también soporta consultas y escritura sobre Hive. Para ello, Spark debe estar configurado para que lea del almacén de Hive correcto y tendremos que usar la sesión de Spark SQL.